# README.md

# Assign ADT window for the 107 pts from previously calculated results approach #2 and manual reviewed cohort with a preference for the latter when there is an overlap

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Assign ADT use](#second-bullet)
* [Assign T recovery status](#third-bullet)

In [51]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

import matplotlib.pyplot as plt
from matplotlib import pyplot
from pylab import *

%matplotlib inline  
import seaborn as sns

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [52]:
#new 107pt list
df1=pd.read_excel("./data/107pt_list.xlsx", header=0)

#manual reviewed cohort 
df2=pd.read_excel("./data/pca_manual_reviewed_cohort_185pts_ADT_windows.xlsx", header=0)

#approach #2 results
df3=pd.read_csv("./data/sean_big_cohort_approach2_561pts_with_adt_window_and_baseline_T_Feb24_2021.csv", header=0)

df1.fillna('', inplace=True)
df1.replace(r'', np.NaN)

df2.fillna('', inplace=True)
df2.replace(r'', np.NaN)
df2[:3]

df3.fillna('', inplace=True)
df3.replace(r'', np.NaN)
df3[:3]

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_time_window_days,Tdate_justbeforeADT,Tvalue_justbeforeADT,baseline avail Status,N_T_dates,N_T_dates_beforeADT
0,2,2009-02-26,2009-09-15,201.0,2009-02-09,274.18,Y,82,20
1,13,2010-06-17,2015-03-17,1734.0,2009-09-18,23.72,Y,23,2
2,25,2013-11-01,2014-02-18,109.0,2013-10-15,31.54,Y,12,1


In [53]:
df2[:3]

,person_id,ADT initail use START,ADT initial use END,ADT duration,Last note date
0,12,2013-05-06,2013-05-06,0,2020-12-18
1,18,2013-11-12,2014-03-13,121,2020-10-29
2,25,2013-08-01,2014-02-18,201,2020-01-22


In [54]:
#first assgin window according to the manual reviewed cohort
#then for the empty lines, assgin them according to the approach #2 
result_pt_list=df1["person_id"].to_list()
manual_pt_list=df2["person_id"].to_list()
approach2_list=df3["person_id"].to_list()

#overlapping btw 107 pts list and  manual reviewed list
overlap1=[x  for x in result_pt_list if x in manual_pt_list]
print(len(overlap1))
remaining=[x for x in result_pt_list if x not in overlap1]
print(len(remaining))

54
53


In [55]:
index_list=df2.columns
index_list

df2.columns=['person_id','ADT_start','ADT_end','ADT_duration', 'Last_note_date']

# Assign ADT use <a class="anchor" id="second-bullet"></a>

In [56]:
result_list=[]

for y in result_pt_list:
    if y in  overlap1:
        df2_pt=df2[(df2["person_id"] == y)]
        #pt_drug_list=df1_pt.cancer_drug.to_list()[0]                  
        person_id=y
        start=df2_pt.ADT_start.to_list()[0] 
        end=df2_pt.ADT_end.to_list()[0] 
        duration=df2_pt.ADT_duration.to_list()[0]         
    if y not in  overlap1:
        df3_pt=df3[(df3["person_id"] == y)]
        #pt_drug_list=df1_pt.cancer_drug.to_list()[0]                  
        person_id=y
        start=df3_pt.inital_ADT_start_date.to_list()[0] 
        end=df3_pt.inital_ADT_end_date.to_list()[0] 
        duration=df3_pt.Inital_ADT_time_window_days.to_list()[0] 
    
    result_per_pt=[person_id,start, end, duration]         
    #print(result_per_pt)    
    result_list.append(result_per_pt)

In [57]:
df_results=pd.DataFrame(result_list)
df_results.columns=(['person_id','ADT_start','ADT_end','ADT_duration'])

In [58]:
df_results[:3]

,person_id,ADT_start,ADT_end,ADT_duration
0,81,2014-04-15,2016-06-09,786.0
1,1257,2014-10-02,2015-07-08,279.0
2,1508,2016-08-03,2017-04-16,256.0


In [59]:
print(shape(df_results)[0])

107


In [60]:
df_results.to_csv("./data/107pts_ADT_windows.csv", encoding='utf-8', index=False)

In [67]:
# T recovery status 300 ng/dL

# Assign T recovery status <a class="anchor" id="third-bullet"></a>

In [62]:
#list of pts without adt
drop_list=[21832,22100, 35343, 4595, 7611, 14278, 14287, 31839, 2063, 7870, 9975, 17580, 25886, 30554, 31492]

In [63]:
#df_results
df_test=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)
df_test.replace(r'', np.NaN)

df1=df_results
df2=df_test
result_status=[]

final_list = [y for y in result_pt_list if y not in drop_list]

for y in final_list:    
    df1_pt=df1[(df1["person_id"] == y)]

    #if(df1_pt.inital_ADT_end_date.values.any()!=""):
    if(pd.notnull(df1_pt['ADT_end'].iloc[0])):
        a=df1_pt['ADT_start'].to_list()[0]
        b=df1_pt['ADT_end'].to_list()[0]
        gap=df1_pt['ADT_duration'].to_list()[0]       
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"

#get the 300
        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=300][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <300][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    #print(y, a, b, gap, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,a, b,gap, tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status.append(result_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [64]:
df_recovery_result300=pd.DataFrame(result_status)
df_recovery_result300.columns=(['person_id', 'inital_ADT_start_date', 'inital_ADT_end_date','Inital_ADT_time_window_days', 'Tdate_rec_norec300', 'Tvalue_rec_norec300', 'Status300',"N_T_dates","N_T_dates_afterADT"])

df_rec300=df_recovery_result300[df_recovery_result300["Status300"]=="Y"]
df_nonrec300=df_recovery_result300[df_recovery_result300["Status300"]=="N"]

In [65]:
df_rec300.to_csv("./data/107pts_ADT_windows_recovery.csv", encoding='utf-8', index=False)
df_nonrec300.to_csv("./data/107pts_ADT_windows_nonrecov.csv", encoding='utf-8', index=False)